This workbook transforms the data from individual 1hz readings into a high 
level overview based on a track. 

This enables a data reduction of 86,400 --> 1 which significantly improves the 
ml loading

Load modules

In [1]:
import os
import datetime
from pathlib import Path
import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

Load the data

In [2]:
path = Path(r'/Users/sr2/OneDrive - University College London/PhD/Research/'
        'Missions/SWARM/Non-Flight Data/Analysis/Mar-22/data/solar_max/')

dir_suffix = '2014'
#pre_classified = str(path) + '/' + dir_suffix +'-data-2022-03-03.csv'
post_classified = str(path) + '/ml_model/SG-filtered_14-15.csv'

def open_all(filename):
    print('Loading data...')
    df = pd.read_csv(filename)
    return df

df = open_all(post_classified)
#df = df.iloc[::10000]
#df.shape

Loading data...


TypeError: read_csv() got an unexpected keyword argument 'index'

Feature Engineering

In [3]:
#Add deltas for moments

df['alt'] = (df.pop('alt')/1000) - 6371

df['ne_5'] = df['Ne'].pct_change(periods=5)
df['ne_10'] = df['Ne'].pct_change(periods=10)
df['ne_20'] = df['Ne'].pct_change(periods=20)

df['te_5'] = df['Te'].pct_change(periods=5)
df['te_10'] = df['Te'].pct_change(periods=10)
df['te_20'] = df['Te'].pct_change(periods=20)

#df['pot_5'] = df['pot'].pct_change(periods=5)
#df['pot_10'] = df['pot'].pct_change(periods=10)
#df['pot_20'] = df['pot'].pct_change(periods=20)

df['ti_5'] = df['Ti'].pct_change(periods=5)
df['ti_10'] = df['Ti'].pct_change(periods=10)
df['ti_20'] = df['Ti'].pct_change(periods=20)

#New time
df['datetime'] = df['date'] +' '+ df['utc']
#date_time = pd.to_datetime(df.pop('datetime'), format='%Y-%m-%d %H:%M:%S')
df=df.dropna()
#df.head()


Add Kp and Ap

In [4]:
#Download kp and ap files
'''
from ftplib import FTP
import pandas as pd

ktp = FTP('ftp.gfz-potsdam.de')
ktp.login()
ktp.cwd('/pub/home/obs/Kp_ap_Ap_SN_F107')
years = [2014,2015,2016,2017,2018,2018,2019,2020,2021]
for i in years:
    filename = 'Kp_ap_'+str(i)+'.txt'
    with open(filename, 'wb' ) as file :
        ktp.retrbinary('RETR %s' % filename, file.write)

        file.close()
    print(f'Downloading data for {i}')

ktp.quit()'''

get_hr = df['utc'].str.slice(stop=2).astype(float)
def round_down(num):
    return num - (num%3)
df['hr'] = get_hr.apply(round_down)

#df_new = pd.merge(df, df_kp, on=['hr','date'])



Add DST & 10.7 dummy values

In [5]:
#Kp
#Round up hours to 3 hours

#dummy data for now
df_len = df_new.shape[0]
dst = np.linspace(-200, 200, num=df_len)
ten_7 = np.linspace(25, 300, num=df_len)
df_new['dst'] = dst
df_new['107'] = ten_7

NameError: name 'df_new' is not defined

Conert time into a seconds. It cannot properly interpret hh:mm:ss

In [ ]:
#Convert time into into a signal
timestamp_s = date_time.map(pd.Timestamp.timestamp)

day = 24*60*60 #hr, min, sec
year = (365.245)*day

df_new['d_sin'] = np.sin(timestamp_s * (2 * np.pi / day))
df_new['d_cos'] = np.cos(timestamp_s * (2 * np.pi / day))
df_new['y_sin'] = np.sin(timestamp_s * (2 * np.pi / year))
df_new['y_cos'] = np.cos(timestamp_s * (2 * np.pi / year))


'''
import tensorflow as tf
fft = tf.signal.rfft(df_new['sg_smooth'])
f_per_dataset = np.arange(0, len(fft))

n_samples_h = len(df_new['sg_smooth'])
hours_per_year = 24*365.2524
years_per_dataset = n_samples_h/(hours_per_year)
f_per_year = f_per_dataset/years_per_dataset

plt.step(f_per_year, np.abs(fft))
plt.xscale('log')
#plt.yscale('log')
plt.ylim(0, 60000)
plt.xlim([0.1, max(plt.xlim())])
plt.xticks([1, 365.2524], labels=['1/Year', '1/day'])
_ = plt.xlabel('Frequency (log scale)')
'''


'''
plt.plot(np.array(df_new['Day sin'])[::86400])
plt.plot(np.array(df_new['Day cos'])[::86400])
plt.xlabel('Time [h]')
plt.title('Time of day signal')'''

In [6]:
#Convert lat and long into radians
df['lat'] = df.pop('lat')*np.pi / 180
df['long'] = df.pop('long')*np.pi / 180

'''
plt.plot(np.array(df['lat'])[::80000])
plt.plot(np.array(df['long'])[::80000])
plt.xlabel('Steps')
plt.title('Lat Long Signal')'''

"\nplt.plot(np.array(df['lat'])[::80000])\nplt.plot(np.array(df['long'])[::80000])\nplt.xlabel('Steps')\nplt.title('Lat Long Signal')"

In [7]:
#df.shape
#remove old values
#pd.set_option('display.max_rows',  None) #or 10 or None
df.describe().transpose()
#kp_df.shape

,count,mean,std,min,25%,50%,75%,max
mlt,1883661.0,17.015606,8.505357,1.203115e-06,19.077634,21.022853,22.329354,2.400000e+01
p_num,1883661.0,1794.278828,985.073704,2.000000e+00,919.000000,1787.000000,2644.000000,3.528000e+03
b_ind,1883661.0,0.036840,0.188369,0.000000e+00,0.000000,0.000000,0.000000,1.000000e+00
b_prob,1883661.0,0.028502,0.157728,0.000000e+00,0.000000,0.000000,0.000000,9.909435e-01
Ne,1883661.0,475121.429140,436702.479968,7.910000e+02,149795.500000,320613.900000,672020.600000,3.114226e+06
Ne_std,1883661.0,0.013891,0.096332,5.691376e-05,0.001027,0.002020,0.004246,1.529460e+01
Ti,1883661.0,1210.592694,157.397160,7.250300e+02,1104.120000,1223.920000,1330.110000,2.178510e+03
Ti_std,1883661.0,0.003036,0.012806,2.232638e-05,0.000391,0.000584,0.000965,6.218901e-01
pot,1883661.0,-0.999392,0.255212,-6.369000e+00,-1.137000,-0.962000,-0.801000,2.336000e+00
pot_std,1883661.0,0.017126,3.917300,0.000000e+00,0.000510,0.000662,0.001027,1.686500e+03


Transform the data from 1hz to statistical overview of a pass. 86k --> 1

In [11]:
def transform_df(df):

    def groupby(df, feat, func_n, rename_col):
        func = getattr(df.groupby(['date','p_num'], as_index=False)[feat], func_n)
        df = func().rename(columns={feat:rename_col}).drop(columns=['date','p_num'])
        
        return df

    #min
    ne_min = df.groupby(['date','p_num'], as_index=False)['Ne'].min().rename(
            columns={'Ne':'ne_min'}) #unique. Because need the date and
    ti_min = groupby(df,'Ti','min','ti_min')
    pot_min = groupby(df,'pot','min','pot_min')
    te_min = groupby(df,'Te','min','te_min')

    #max
    ne_max = groupby(df,'Ne', 'max', 'ne_max')
    ti_max = groupby(df, 'Ti','max', 'ti_max')
    pot_max = groupby(df,'pot','max','pot_max')
    te_max = groupby(df,'Te','max','te_max')

    #mean
    ne_mean = groupby(df, 'Ne', 'mean', 'ne_mean')
    ti_mean = groupby(df, 'Ti', 'mean', 'ti_mean')
    pot_mean = groupby(df,'pot','mean','pot_mean')
    te_mean = groupby(df,'Te','mean','te_mean')

    #stddev
    ne_std = groupby(df, 'Ne', 'std', 'ne_std')
    ti_std = groupby(df, 'Ti', 'std', 'ti_std')
    pot_std = groupby(df,'pot','std','pot_std')
    te_std = groupby(df,'Te','std','te_std')

    #delta 
    ne_5 = groupby(df, 'ne_5', 'max', 'ne_5')
    ne_10 = groupby(df, 'ne_10', 'max', 'ne_10')
    ne_20 = groupby(df, 'ne_20', 'max', 'ne_20')

    te_5 = groupby(df, 'te_5', 'max', 'te_5')
    te_10 = groupby(df, 'te_10', 'max', 'te_10')
    te_20 = groupby(df, 'te_20', 'max', 'te_20')

    ti_5 = groupby(df, 'ti_5', 'max', 'ti_5')
    ti_10 = groupby(df, 'ti_5', 'max', 'ti_10')
    ti_20 = groupby(df, 'ti_5', 'max', 'ti_20')

    #"Header" info
    #normal
    lat = groupby(df, 'lat', 'mean', 'lat')
    lon = groupby(df, 'long', 'mean', 'long')
    mlt = groupby(df, 'mlt', 'mean', 'mlt')
    alt = groupby(df, 'alt', 'mean', 'alt')
    hr = groupby(df, 'hr','min','hr')

    #fequencies
    #d_sin = groupby(df, 'd_sin', 'mean', 'd_sin')
    #d_cos = groupby(df, 'd_cos', 'mean', 'd_cos')
    #y_sin = groupby(df, 'y_sin', 'mean', 'y_sin')
    #y_cos = groupby(df, 'y_cos', 'mean', 'y_cos')


    #EPB
    #ibi_epb = groupby(df, 'b_ind', 'max', 'IBI')
    mssl_epb = groupby(df, 'sg_smooth','max','epb')
    #ibi_size = groupby(df, 'b_ind', 'sum', 'ibi_size')
    mssl_size = groupby(df, 'sg_smooth', 'sum', 'epb_size')
    
    def epb_size_cal(df,col):
        df[col] = df[col] * 7.7
        return df
    
    #ibi_size = epb_size_cal(ibi_size, 'ibi_size')
    mssl_size = epb_size_cal(mssl_size, 'epb_size')

    #Non-SWARM inputs
    #Kp = groupby(df, 'Kp', 'mean', 'Kp')
    #ap = groupby(df, 'Kp', 'mean', 'ap')
    #dst = groupby(df, 'dst', 'mean', 'dst')
    #ten_7 = groupby(df, '107', 'mean', '107')

    df = pd.concat([
        lat, lon, mlt, alt, hr,
        #d_sin, d_cos, y_sin, y_cos,
        ne_min, ne_max, ne_mean, ne_std,
        pot_min, pot_max, pot_mean, pot_std,
        ti_min, ti_max, ti_mean, ti_std,
        te_min, te_max, te_mean, te_std,
        ne_5, ne_10, ne_20, te_5, te_10, te_20, ti_5, ti_10, ti_20,
        #Kp, ap,
        #ne_del, pot_del, ti_del,
        #ibi_epb, ibi_size, 
        mssl_size, mssl_epb], axis=1)

    df = df.sort_values(by=['date','p_num'], ascending=[True,True])

    return df

df_gb = transform_df(df)

df_gb.head(323).transpose()

,0,1,2,3,4,5,6,7,8,9,...,313,314,315,316,317,318,319,320,321,322
lat,0.490663,-0.0753,0.112872,0.139017,0.165121,0.206794,0.160862,0.138995,0.050936,0.155043,...,0.074398,-0.520799,0.220343,-0.390312,-0.011672,-0.011909,-0.012144,-0.012267,-0.011601,-0.012403
long,-1.297054,-1.30693,-1.716866,-2.129635,-2.542287,-2.954857,2.914247,2.500679,2.085916,1.674614,...,0.688086,0.679469,0.279311,0.269492,-0.135748,-0.547047,-0.958348,-1.369676,-1.781011,-2.192348
mlt,20.059348,19.978584,19.926068,19.937049,20.033945,20.117453,20.160647,20.181363,20.281656,20.304382,...,0.633662,0.366019,0.608661,0.463792,0.545417,0.642826,0.726521,0.630484,0.528442,1.688757
alt,486.821497,492.141427,490.587296,490.422942,490.143321,489.715827,490.088091,490.184718,491.013949,490.142506,...,472.417369,477.145809,471.024695,476.254642,472.937528,472.823132,472.788361,472.911454,473.051667,473.128348
hr,0.0,0.0,0.0,3.0,3.0,6.0,9.0,9.0,12.0,12.0,...,21.0,21.0,21.0,21.0,0.0,0.0,3.0,3.0,6.0,6.0
date,2014-02-01,2014-02-01,2014-02-01,2014-02-01,2014-02-01,2014-02-01,2014-02-01,2014-02-01,2014-02-01,2014-02-01,...,2014-09-09,2014-09-09,2014-09-09,2014-09-09,2014-09-10,2014-09-10,2014-09-10,2014-09-10,2014-09-10,2014-09-10
p_num,2838,2839,2841,2843,2845,2847,2849,2851,2853,2855,...,1045,1046,1047,1048,1988,1990,1992,1994,1996,1998
ne_min,93003.3,97846.7,53225.1,47012.8,41987.1,49220.7,50843.4,76118.0,64549.4,77057.2,...,84771.9,64762.6,175640.1,50801.2,46761.8,29058.0,62048.0,137333.6,104581.6,128889.4
ne_max,114583.3,1585525.0,1195532.7,724227.9,704825.6,727323.0,912141.3,989496.0,1316978.6,1126839.7,...,664052.0,83010.0,1205107.8,139923.1,581372.2,408091.7,418266.4,311417.1,286887.5,450991.7
ne_mean,109189.571676,685650.681408,428758.208346,298335.639085,251495.84888,306559.573775,467342.984273,478073.464507,653904.315138,723742.541404,...,257054.592588,72155.719375,495826.156794,63349.46285,214181.377715,130200.778371,224902.505712,234565.027528,200867.185206,213230.77045


Get Kp and Ap

In [14]:
years = [2014,2015,2016,2017,2018,2018,2019,2020,2021]
listing = []
for i in years:
    filename = 'Kp_ap_'+str(i)+'.txt'
    df_kp = pd.read_csv(filename, on_bad_lines='skip', sep=" ")
    df_kp = df_kp.iloc[18:].reset_index().drop(columns=['level_0','index','ap','index.1','three-hour','interval)','distributes','the','Kp','per'])
    df_kp = df_kp.rename(columns={'#':'yy','PURPOSE:':'mm','This':'dd','file':'hr','and':'Kp','(one':'ap1','line':'ap2'})
    df_kp['date'] = df_kp['yy'] +'-'+ df_kp['mm'] +'-'+ df_kp['dd']
    df_kp = df_kp.drop(['yy', 'mm','dd'], axis=1)
    df_kp['hr']= df_kp['hr'].astype(float)

    df_kp['ap2']= df_kp['ap2'].mask(df_kp['ap2'] == 1, np.nan)
    df_kp['ap']=df_kp['ap2'].fillna(df_kp['ap1'])
    df_kp = df_kp.drop(columns=['ap1','ap2'])

    listing.append(df_kp)

df_kp = pd.concat(listing)

df_gb = df_gb.merge(df_kp,on=['hr','date'])
df_gb

,lat,long,mlt,alt,hr,date,p_num,ne_min,ne_max,ne_mean,...,te_20,ti_5,ti_10,ti_20,epb_size,epb,Kp_x,ap_x,Kp_y,ap_y
0,0.490663,-1.297054,20.059348,486.821497,0.0,2014-02-01,2838,93003.3,114583.3,109189.571676,...,0.745595,0.003443,0.003443,0.003443,0.0,0,0.000,0,0.000,0
1,-0.075300,-1.306930,19.978584,492.141427,0.0,2014-02-01,2839,97846.7,1585525.0,685650.681408,...,1.833173,0.434945,0.434945,0.434945,1809.5,1,0.000,0,0.000,0
2,0.112872,-1.716866,19.926068,490.587296,0.0,2014-02-01,2841,53225.1,1195532.7,428758.208346,...,1.043940,0.708490,0.708490,0.708490,531.3,1,0.000,0,0.000,0
3,0.139017,-2.129635,19.937049,490.422942,3.0,2014-02-01,2843,47012.8,724227.9,298335.639085,...,1.878037,0.403777,0.403777,0.403777,215.6,1,0.000,0,0.000,0
4,0.165121,-2.542287,20.033945,490.143321,3.0,2014-02-01,2845,41987.1,704825.6,251495.848880,...,1.521660,0.564149,0.564149,0.564149,377.3,1,0.000,0,0.000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2583,0.015324,1.801998,23.750744,459.089591,15.0,2015-10-27,566,72713.1,577715.5,214097.386364,...,0.054118,0.441130,0.441130,0.441130,0.0,0,0.333,2,0.333,2
2584,0.010879,1.391996,23.631268,458.990147,18.0,2015-10-27,568,63823.2,371674.0,186742.393052,...,0.070466,0.407273,0.407273,0.407273,0.0,0,1.000,4,1.000,4
2585,0.010929,0.981953,23.571467,458.861316,18.0,2015-10-27,570,86346.2,474384.1,222562.897556,...,0.055519,0.441107,0.441107,0.441107,0.0,0,1.000,4,1.000,4
2586,-0.005692,0.572214,23.529092,458.953587,21.0,2015-10-27,572,92794.7,455566.1,186144.310542,...,0.088271,0.403816,0.403816,0.403816,0.0,0,2.000,7,2.000,7


Get IGRF Data

In [ ]:
def get_igrf(col):
    import requests
    import xml.etree.ElementTree as ET

    lat = col['lat']
    lon = col['long']
    alt = col['alt']
    date = col['date']
    
    path = 'https://geomag.bgs.ac.uk/web_service/GMModels/igrf/13/?latitude='+str(lat)+'&longitude='+str(lon)+'&altitude='+str(alt)+'&date='+str(date)+'&format=xml'
    
    response = requests.get(path)

    root = ET.fromstring(response.content)
    date = root[1].text

    for coors in root.findall('coordinates'):
        _lat = coors.find('latitude').text
        _lon = coors.find('longitude').text

    for fields in root.findall('field-value'):
        _ti = fields.find('total-intensity').text
        _dec = fields.find('declination').text
        _inc = fields.find('inclination').text
        _ni = fields.find('north-intensity').text
        _ei = fields.find('east-intensity').text
        _vi = fields.find('vertical-intensity').text
        _hi = fields.find('horizontal-intensity').text

    for secular in root.findall('secular-variation'):
        s_ti = secular.find('total-intensity').text
        s_dec = secular.find('declination').text
        s_inc = secular.find('inclination').text
        s_ni = secular.find('north-intensity').text
        s_ei = secular.find('east-intensity').text
        s_vi = secular.find('vertical-intensity').text
        s_hi = secular.find('horizontal-intensity').text

    #return pd.DataFrame({'ti':[_ti],'dec':_dec})

    return pd.Series([_ti, _dec, _inc, _ni, _ei, _vi, _hi,
            s_ti, s_dec, s_inc, s_ni, s_ei, s_vi, s_hi])

df_gb[['ti','dec','inc','ni','ei','vi','hi','s_ti','s_dec','s_inc','s_ni','s_ei','s_vi','s_hi']]= df_gb.apply(get_igrf, axis=1)


In [15]:
pd.set_option('display.max_rows',  None) #or 10 or None
#df_gb = df_gb.drop(columns=['date','p_num'])
df_gb.replace([np.inf, -np.inf], np.nan, inplace=True)
df_gb.describe().transpose()


,count,mean,std,min,25%,50%,75%,max
lat,2588.0,0.003657,0.267163,-0.608673,-0.089444,0.006816,1.167749e-01,6.084387e-01
long,2588.0,0.013620,1.784640,-3.133908,-1.489253,0.007232,1.548868e+00,3.131544e+00
mlt,2588.0,17.158374,7.980877,0.053989,17.738061,20.879602,2.218727e+01,2.396825e+01
alt,2588.0,468.623628,6.901139,452.895547,464.469014,468.377113,4.723511e+02,4.963093e+02
hr,2588.0,10.476816,6.966639,0.000000,3.000000,12.000000,1.800000e+01,2.100000e+01
p_num,2588.0,1804.603941,982.346946,2.000000,984.750000,1815.500000,2.646000e+03,3.528000e+03
ne_min,2588.0,130177.749845,114410.696728,791.000000,69916.950000,103024.000000,1.487164e+05,1.251022e+06
ne_max,2588.0,983721.860510,686669.932042,32797.900000,382492.775000,891595.750000,1.460070e+06,3.114226e+06
ne_mean,2588.0,445029.523441,273851.082441,25104.501911,209193.139821,423828.500434,6.400707e+05,1.934457e+06
ne_std,2588.0,251134.617804,201422.727254,105.617502,69607.963569,221795.286667,3.967143e+05,8.858817e+05


Address outliers

In [ ]:
#Address long tails
df_gb['te_max'] = df_gb['te_max'].where(df_gb['te_max'] < 5000, df_gb['te_max'].mean())
df_gb['te_std'] = df_gb['te_std'].where(df_gb['te_std'] < 1000, df_gb['te_std'].mean())
df_gb['ne_5'] = df_gb['ne_5'].where(df_gb['ne_5'] < 50, 50)
df_gb['ne_10'] = df_gb['ne_10'].where(df_gb['ne_10'] < 100, 100)
df_gb['ne_20'] = df_gb['ne_20'].where(df_gb['ne_20'] < 200, 200)
df_gb['te_5'] = df_gb['te_5'].where(df_gb['te_5'] < 5, 5)
df_gb['te_10'] = df_gb['te_10'].where(df_gb['te_10'] < 3, 3)
df_gb['te_20'] = df_gb['te_20'].where(df_gb['te_20'] < 3, 3)


Visualise the Data

In [ ]:
'''
plt.figure(figsize=(15,9), dpi=100)
mask = np.triu(np.ones_like(corr_df, dtype=bool))
sns.heatmap(corr_df, center=0, mask=mask, cbar_kws={"shrink": .7},
        square=False, linewidths=0.5, annot=True, fmt=".1f")'''

In [ ]:
#Violin or box plot
df_std = df_gb.std()
df_mean = df_gb.mean()
df_violin = (df_gb - df_mean) / df_std
df_violin = df_violin.melt(var_name='Column', value_name='Normalized')
plt.figure(figsize=(15, 5), dpi=150)
plt.rc('font', size=14) 
ax = sns.boxplot(x='Column', y='Normalized', data=df_violin)
_ = ax.set_xticklabels(df_gb.keys(), rotation=90)
plt.xlabel(' ')

Export ready for ML

In [ ]:
df_name = 'LSTM_53-feats.csv'
df_gb.to_csv(df_name, index=False, header=True)
print('df exported')